Playing around with data processing for Text as Data project

In [1]:
import pandas as pd

In [2]:
daily_loc = './Congress_Dataset/hein-daily/'

In [11]:
# try with most recent congress session contained in data
descr = 'descr_114.txt'
speeches = 'speeches_114.txt'
speaker_data = '114_SpeakerMap.txt'

# try reading in data
metadata = pd.read_csv(daily_loc+descr, sep='|')
# convert date column to datetime
metadata ['date']= pd.to_datetime(metadata ['date'], format='%Y%m%d')

In [14]:
metadata.head()

,speech_id,chamber,date,number_within_file,speaker,first_name,last_name,state,gender,line_start,line_end,file,char_count,word_count
0,1140000001,None,2015-01-06,1,The CLERK,Unknown,Unknown,Unknown,Special,61,64,01062015.txt,107,16
1,1140000002,None,2015-01-06,2,The CLERK,Unknown,Unknown,Unknown,Special,71,96,01062015.txt,840,136
2,1140000003,H,2015-01-06,3,The CLERK,Unknown,Unknown,Unknown,Special,746,750,01062015.txt,93,14
3,1140000004,H,2015-01-06,4,The CLERK,Unknown,Unknown,Unknown,Special,753,774,01062015.txt,597,92
4,1140000005,H,2015-01-06,5,The CLERK,Unknown,Unknown,Unknown,Special,778,783,01062015.txt,78,14


In [5]:
df_speech = pd.read_csv(daily_loc+speeches, sep='|', encoding= 'unicode_escape', on_bad_lines='skip')

In [17]:
print(df_speech.shape)
df_speech.head()

(112287, 2)


In [26]:
# add date to df_speech from metadata
df_speech = df_speech.merge(metadata[['speech_id', 'date']], on='speech_id')
print(df_speech.shape)
df_speech.head()

(112287, 3)


,speech_id,speech,date
0,1140000001,The Representativeselect and their guests will...,2015-01-06
1,1140000002,As directed by law. the Clerk of the House has...,2015-01-06
2,1140000003,Four hundred and one Represent ativeselect hav...,2015-01-06
3,1140000004,Credentials. regular in form. have been receiv...,2015-01-06
4,1140000005,The Clerk is in receipt of a letter from the H...,2015-01-06


In [27]:
df_speakers = pd.read_csv(daily_loc+speaker_data, sep='|', encoding= 'unicode_escape')

In [28]:
df_speakers.head()

,speakerid,speech_id,lastname,firstname,chamber,state,gender,party,district,nonvoting
0,114120480,1140000007,MCMORRIS RODGERS,CATHY,H,WA,F,R,5.0,voting
1,114118560,1140000009,BECERRA,XAVIER,H,CA,M,D,34.0,voting
2,114121890,1140000011,MASSIE,THOMAS,H,KY,M,R,4.0,voting
3,114122500,1140000013,BRIDENSTINE,JIM,H,OK,M,R,1.0,voting
4,114120780,1140000017,PELOSI,NANCY,H,CA,F,D,12.0,voting


In [29]:
# get democratic and republican speakers
dem_speakers = df_speakers[df_speakers['party'] == 'D']
rep_speakers = df_speakers[df_speakers['party'] == 'R']
print(dem_speakers.shape)
print(rep_speakers.shape)

(29293, 10)
(38264, 10)


In [30]:
# get from a specific state
dem_CA_speakers = dem_speakers[dem_speakers['state'] == 'CA']
print(dem_CA_speakers.shape)
dem_CA_speakers.head()

(4746, 10)


,speakerid,speech_id,lastname,firstname,chamber,state,gender,party,district,nonvoting
1,114118560,1140000009,BECERRA,XAVIER,H,CA,M,D,34.0,voting
4,114120780,1140000017,PELOSI,NANCY,H,CA,F,D,12.0,voting
8,114118560,1140000024,BECERRA,XAVIER,H,CA,M,D,34.0,voting
9,114118560,1140000026,BECERRA,XAVIER,H,CA,M,D,34.0,voting
10,114118560,1140000029,BECERRA,XAVIER,H,CA,M,D,34.0,voting


In [31]:
# try to get all speeches by nancy pelosi
pelosi_id = 114120780
# get all speech_ids for nancy pelosi
pelosi_speech_ids = dem_CA_speakers[dem_CA_speakers['speakerid'] == pelosi_id]
# pelosi_speeches = df_speech[df_speech['speech_id']]
print(pelosi_speech_ids.shape)
pelosi_speech_ids.head()

(85, 10)


,speakerid,speech_id,lastname,firstname,chamber,state,gender,party,district,nonvoting
4,114120780,1140000017,PELOSI,NANCY,H,CA,F,D,12.0,voting
47,114120780,1140000099,PELOSI,NANCY,H,CA,F,D,12.0,voting
485,114120780,1140001022,PELOSI,NANCY,H,CA,F,D,12.0,voting
601,114120780,1140001269,PELOSI,NANCY,H,CA,F,D,12.0,voting
790,114120780,1140001471,PELOSI,NANCY,H,CA,F,D,12.0,voting


In [32]:
pelosi_speeches = df_speech[df_speech['speech_id'].isin(pelosi_speech_ids['speech_id'])]
pelosi_speeches['speakerid'] = pelosi_id
print(pelosi_speeches.shape)
pelosi_speeches.head()

(85, 4)


/var/folders/fy/r2tc5r9d7tq2nh7g2jhy2ydm0000gn/T/ipykernel_15350/2400294442.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pelosi_speeches['speakerid'] = pelosi_id


,speech_id,speech,date,speakerid
16,1140000017,My colleagues of the United States House of Re...,2015-01-06,114120780
98,1140000099,Mr. Speaker. I thank the gentlelady for yieldi...,2015-01-06,114120780
1021,1140001022,Mr. Speaker. I know at some point the House wi...,2015-01-08,114120780
1268,1140001269,Mr. Speaker. I come to the floor to celebrate ...,2015-01-09,114120780
1470,1140001471,"Section 5: ""The Congress shall have the power ...",2015-01-09,114120780


TODO:
- add date info - DONE
- get all speeches from a specific party

In [35]:
# want dataframe with:
# speaker_id, speech_id, speech, date, party, state, first_name, last_name
print(df_speakers.columns)
print(metadata.columns)
print(df_speech.columns)

Index(['speakerid', 'speech_id', 'lastname', 'firstname', 'chamber', 'state',
       'gender', 'party', 'district', 'nonvoting'],
      dtype='object')
Index(['speech_id', 'chamber', 'date', 'number_within_file', 'speaker',
       'first_name', 'last_name', 'state', 'gender', 'line_start', 'line_end',
       'file', 'char_count', 'word_count'],
      dtype='object')
Index(['speech_id', 'speech', 'date'], dtype='object')


In [42]:
print('original df_speech size:', df_speech.size)
print('original df_speakers size:', df_speakers.size)

original df_speech size: 336861
original df_speakers size: 679710


In [47]:
# just need to combine df_speech with df_speakers??
df_1 = df_speakers[['speakerid', 'speech_id', 'lastname', 'firstname', 'state', 'party']]
df_all = df_1.merge(df_speech, on='speech_id')
print(df_all.shape) # we lose some data but that's okay because we need party and state info, etc.
df_all.head()

(67970, 8)


,speakerid,speech_id,lastname,firstname,state,party,speech,date
0,114120480,1140000007,MCMORRIS RODGERS,CATHY,WA,R,RODGERS. Madam Clerk. it is an honor to addres...,2015-01-06
1,114118560,1140000009,BECERRA,XAVIER,CA,D,Madam Clerk. first I would like to recognize e...,2015-01-06
2,114121890,1140000011,MASSIE,THOMAS,KY,R,Madam Clerk. I present for election to the off...,2015-01-06
3,114122500,1140000013,BRIDENSTINE,JIM,OK,R,Madam Clerk. I present for the election of the...,2015-01-06
4,114120780,1140000017,PELOSI,NANCY,CA,D,My colleagues of the United States House of Re...,2015-01-06


In [48]:
# write this to a separate file
df_all.to_csv('Data/processed_data_114.csv')

## Processing Data

In [58]:
# write function that does this, and then repeat for all files!
import numpy as np

def process_data(session_num):
    print('processing ' + session_num)
    # folder and file names
    daily_loc = './Congress_Dataset/hein-daily/'
    descr = 'descr_' + session_num + '.txt'
    speeches = 'speeches_' + session_num + '.txt'
    speaker_data = session_num + '_SpeakerMap.txt'
    
    # read in 3 files
    print('reading descr')
    metadata = pd.read_csv(daily_loc+descr, sep='|', dtype={'speech_id': object})
    print('reading SpeakerMap')
    df_speakers = pd.read_csv(daily_loc+speaker_data, sep='|', encoding= 'unicode_escape', 
                              dtype={'speech_id': object})
    print('reading speeches')
    df_speech = pd.read_csv(daily_loc+speeches, sep='|', encoding= 'unicode_escape', 
                            encoding_errors = 'ignore',on_bad_lines='skip',
                           dtype={'speech_id': object, 'speech': object})
    
    # convert date column to datetime
    metadata ['date']= pd.to_datetime(metadata ['date'], format='%Y%m%d')
    
    # add date to df_speech from metadata
    df_speech = df_speech.merge(metadata[['speech_id', 'date']], on='speech_id')
    
    # merge to get necessary columns only
    df_1 = df_speakers[['speakerid', 'speech_id', 'lastname', 'firstname', 'state', 'party']]
    df_all = df_1.merge(df_speech, on='speech_id')
    
    # write to csv
    file_name = 'Data/processed_data_' + session_num + '.csv'
    df_all.to_csv(file_name)
    
    # return df
    return df_all
    
# process all sessions!
ids = ['097', '098', '099', '100', '101', '102', '103', '104', '105', 
       '106', '107', '108', '109', '110', '111', '112', '113', '114']
for i in ids:
    process_data(i)

processing 097
reading descr
reading SpeakerMap
reading speeches
processing 098
reading descr
reading SpeakerMap
reading speeches
processing 099
reading descr
reading SpeakerMap
reading speeches
processing 100
reading descr
reading SpeakerMap
reading speeches
processing 101
reading descr
reading SpeakerMap
reading speeches
processing 102
reading descr
reading SpeakerMap
reading speeches
processing 103
reading descr
reading SpeakerMap
reading speeches
processing 104
reading descr
reading SpeakerMap
reading speeches
processing 105
reading descr
reading SpeakerMap
reading speeches
processing 106
reading descr
reading SpeakerMap
reading speeches
processing 107
reading descr
reading SpeakerMap
reading speeches
processing 108
reading descr
reading SpeakerMap
reading speeches
processing 109
reading descr
reading SpeakerMap
reading speeches
processing 110
reading descr
reading SpeakerMap
reading speeches
processing 111
reading descr
reading SpeakerMap
reading speeches
processing 112
reading de